<a href="https://colab.research.google.com/github/vikram759/hand_digit_classifier/blob/main/handwritten_digit_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [5]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=10,
    zoom_range=0.1,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)


In [6]:
def create_model():
    inputs = Input(shape=(28, 28, 1))
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
model = create_model()
early_stop = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)
model.fit(datagen.flow(X_train, y_train_one_hot, batch_size=64),
                    steps_per_epoch=len(X_train) // 64,
                    epochs=5,
                    validation_data=(X_test, y_test_one_hot),
                    callbacks=[early_stop, reduce_lr])
#validation accuracy 98.6% after 5th epoch

Epoch 1/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 32s 29ms/step - accuracy: 0.7154 - loss: 0.9595 - val_accuracy: 0.9775 - val_loss: 0.0705 - learning_rate: 0.0010
Epoch 2/5
  1/937 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.9219 - loss: 0.3304

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step - accuracy: 0.9219 - loss: 0.3304 - val_accuracy: 0.9763 - val_loss: 0.0730 - learning_rate: 0.0010
Epoch 3/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - accuracy: 0.9120 - loss: 0.2987 - val_accuracy: 0.9834 - val_loss: 0.0492 - learning_rate: 0.0010
Epoch 4/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 901us/step - accuracy: 0.9062 - loss: 0.3166 - val_accuracy: 0.9831 - val_loss: 0.0518 - learning_rate: 0.0010
Epoch 5/5
937/937 ━━━━━━━━━━━━━━━━━━━━ 43s 26ms/step - accuracy: 0.9376 - loss: 0.2129 - val_accuracy: 0.9863 - val_loss: 0.0441 - learning_rate: 0.0010
